### Understanding the data

In [ ]:
import json
from IPython.display import JSON
import pprint
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
with open('flickr8k/dataset.json', 'r') as f:
    json_data = json.load(f)

In [ ]:
json_data.keys()

In [ ]:
json_data['dataset']

In [ ]:
json_data['images'][0].keys()

In [ ]:
pprint.pprint(json_data['images'][0])

### Ground truth image descriptions

In [ ]:
def get_gt_image_descriptions(json_data):
    descriptions = defaultdict(list)
    for jd in json_data['images']:
        fn = jd['filename'].split('.')[0]
        for s in jd['sentences']:
            descriptions[fn].append(s['raw'])
    return descriptions

In [ ]:
descriptions = get_gt_image_descriptions(json_data)
sample_image_id = np.random.choice(list(descriptions.keys()))
print('\n'.join(descriptions[sample_image_id]))

### Data cleaning

In [ ]:
import string

In [ ]:
string.punctuation

In [ ]:
def clean_descriptions(descriptions):
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [w.lower() for w in desc]
            # remove punctuation
            desc = [w.translate(table) for w in desc]
            # remove 'a' and 's'
            desc = [w for w in desc if len(w) > 1]
            # remove tokens with numbers in them
            desc = [w for w in desc if w.isalpha()]
            # store as string
            desc_list[i] = ' '.join(desc)
    return descriptions

In [ ]:
clean_descriptions = clean_descriptions(descriptions)

In [ ]:
clean_descriptions[sample_image_id]

In [ ]:
all_descriptions = [len(clean_descriptions[key]) for key in clean_descriptions.keys()]
print(sum(all_descriptions))

### Create Vocabulary

In [ ]:
def create_vocabulary(clean_descriptions):
    #  remove duplicate words (set of unique words)
    vocabulary = set()
    for key in clean_descriptions.keys():
        [vocabulary.update(d.split()) for d in clean_descriptions[key]]
    return vocabulary

In [ ]:
vocabulary = create_vocabulary(clean_descriptions)

In [ ]:
print('vocabulary size:', len(vocabulary))

In [ ]:
def save_descriptions(clean_descriptions, filename):
    lines = []
    for key, desc_list in clean_descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    with open(filename, 'w') as f:
        for line in lines:
            f.writelines(line)
            f.writelines('\n')

In [ ]:
save_descriptions(clean_descriptions, 'descriptions.txt')

### Train and Test Split

In [ ]:
train_clean_desc_keys, test_clean_desc_keys = train_test_split(list(clean_descriptions.keys()))
print('train size:', len(train_clean_desc_keys))
print('test size:', len(test_clean_desc_keys))

In [ ]:
def load_train_clean_descriptions(train_clean_desc_keys, filename):
    train_clean_descriptions = {}
    with open(filename, 'r') as f:
        for line in f:
            line = line.rstrip()
            tokens = line.split()
            image_id, image_desc = tokens[0], tokens[1:]
            # skip images not in the train set
            if image_id in train_clean_desc_keys:
                if not image_id in train_clean_descriptions:
                    train_clean_descriptions[image_id] = []
                # add start and end token
                desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
                train_clean_descriptions[image_id].append(desc)
    return train_clean_descriptions

In [ ]:
train_descriptions = load_train_clean_descriptions(train_clean_desc_keys, 'descriptions.txt')

In [ ]:
print('Descriptions: train={}'.format(len(train_descriptions)))

In [ ]:
print(train_descriptions[sample_image_id])